In [1]:
import csv
import json

def parse_slots(slot_string):
    slots_by_day = {}
    for item in slot_string.split(','):
        if '_' not in item:
            continue
        day_part, time_part = item.strip().split('_')
        start, end = time_part.split('-')
        day = day_part.capitalize()
        if day not in slots_by_day:
            slots_by_day[day] = []
        slots_by_day[day].append({"start": start, "end": end})
    return slots_by_day

def convert_csv_to_json(csv_path, json_path):
    instructors = []

    with open(csv_path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            instructor = {
                "id": f"I{int(row['instructor_id'])}",
                "name": row["name"],
                "courses": [c.strip() for c in row["qualified_courses"].split(',')],
                "availability": parse_slots(row["available_slots"])
            }
            instructors.append(instructor)

    with open(json_path, 'w', encoding='utf-8') as jsonfile:
        json.dump(instructors, jsonfile, indent=4)

# Example usage:
convert_csv_to_json("instructorall.csv", "instructors.json")

In [3]:
#  duplicate teacher managed version:
import csv
import json
from collections import defaultdict

def parse_slots(slot_string):
    slots_by_day = defaultdict(list)
    for item in slot_string.split(','):
        if '_' not in item:
            continue
        day_part, time_part = item.strip().split('_')
        start, end = time_part.split('-')
        day = day_part.capitalize()
        slots_by_day[day].append({"start": start, "end": end})
    return dict(slots_by_day)

def merge_availability(avail1, avail2):
    merged = defaultdict(list)
    for day in set(avail1.keys()).union(avail2.keys()):
        merged[day].extend(avail1.get(day, []))
        merged[day].extend(avail2.get(day, []))
    return dict(merged)

def convert_csv_to_json_with_merge(csv_path, json_path):
    merged_data = {}

    with open(csv_path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            name = row['name'].strip()
            courses = [c.strip() for c in row['qualified_courses'].split(',')]
            availability = parse_slots(row['available_slots'])

            if name in merged_data:
                # Merge courses and availability
                existing = merged_data[name]
                existing['courses'] = sorted(set(existing['courses'] + courses))
                existing['availability'] = merge_availability(existing['availability'], availability)
            else:
                merged_data[name] = {
                    "id": f"I{len(merged_data)+1}",
                    "name": name,
                    "courses": courses,
                    "availability": availability
                }

    # Write merged data to JSON
    with open(json_path, 'w', encoding='utf-8') as jsonfile:
        json.dump(list(merged_data.values()), jsonfile, indent=4)

# Example usage
convert_csv_to_json_with_merge("instructorall.csv", "instructors_merged.json")
